In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re

In [2]:
# Ruta del archivo .pkl
file_path = r'C:\Users\Usuario\Desktop\HENRY\PROYECTO FINAL\Proyecto Final\datasets\Yelp\business.pkl'

# Cargar el archivo .pkl
df_business_yelp_base = pd.read_pickle(file_path)

In [3]:
df_business_y = df_business_yelp_base

In [ ]:
df_business_y